# Advanced Submission Notebook

In this notebook we attempt a more advanced approach. Here we generate essays that try to “exploit” the judging system by mixing contrasting and contradictory sentence templates. We use a set of diverse sentence templates that reference the topic in different ways. The sentences are randomized so that each generated essay is non-repetitive and exhibits internal variance. This could lead the multiple LLM judges to return more divergent scores.

In [1]:
import pandas as pd
import random

We now load the test set from `test.csv`. As before, we expect an `id` column and (optionally) a `topic` column. If a topic is missing, we default to "general discussion."

In [2]:
# Load the test dataset
test_df = pd.read_csv("/kaggle/input/llms-you-cant-please-them-all/test.csv")
if 'topic' not in test_df.columns:
    test_df['topic'] = "general discussion"

Next, we define an advanced essay generator. It uses a collection of sentence templates that embed the topic and juxtapose positive and negative assessments. We randomize the order of the sentences and—if the essay does not reach about 100 words—we add an extra filler sentence. We also allow an optional seed to vary the generation per test row.

In [3]:
def generate_advanced_essay(topic, seed=None):
    """
    Generates an essay with approximately 100 words using multiple sentence templates.
    The sentences are randomized and designed to create internal contradictions,
    potentially causing the LLM judges to disagree.
    """
    if seed is not None:
        random.seed(seed)
        
    # Define a set of sentence templates.
    templates = [
        "In contemplating {topic}, one cannot overlook the paradox of simplicity and complexity that defines its essence.",
        "The juxtaposition of vibrant optimism and subtle despair encapsulates the ambiguous nature of {topic}.",
        "While some may argue that {topic} embodies the pinnacle of creativity, others contend it is a mere facade of ingenuity.",
        "An unexpected twist in the narrative of {topic} reveals layers of both profound insight and triviality.",
        "At first glance, {topic} appears to be an exemplar of meticulous design, yet it conceals contradictions that perplex the observer.",
        "The inherent duality of {topic}—marked by both brilliance and mediocrity—challenges conventional judgment.",
        "Each facet of {topic} oscillates between admiration and skepticism, leaving no room for a singular interpretation.",
        "Ultimately, {topic} stands as a testament to the unpredictable interplay between artistic expression and critical scrutiny."
    ]
    
    # Shuffle the templates and select a subset.
    random.shuffle(templates)
    selected_templates = templates[:5]  # Choose 5 distinct sentences.
    
    # Create the essay text by formatting each template with the topic.
    essay = " ".join([template.format(topic=topic) for template in selected_templates])
    
    # Count the words and append extra filler if necessary.
    words = essay.split()
    # Append filler until we have at least 100 words.
    while len(words) < 100:
        extra_sentence = "This exploration defies categorization and invites both praise and critique from diverse perspectives."
        essay += " " + extra_sentence
        words = essay.split()
    
    # If the essay is too long (say, more than 110 words), trim it.
    if len(words) > 110:
        essay = " ".join(words[:110])
    
    return essay

# Quick test on a sample topic:
print(generate_advanced_essay("the evolution of art", seed=42))

An unexpected twist in the narrative of the evolution of art reveals layers of both profound insight and triviality. At first glance, the evolution of art appears to be an exemplar of meticulous design, yet it conceals contradictions that perplex the observer. Each facet of the evolution of art oscillates between admiration and skepticism, leaving no room for a singular interpretation. Ultimately, the evolution of art stands as a testament to the unpredictable interplay between artistic expression and critical scrutiny. While some may argue that the evolution of art embodies the pinnacle of creativity, others contend it is a mere facade of ingenuity.


We now apply our advanced essay generator to each test example. To add more variety per example, we use a seed that depends on the test row index. Finally, we output the submission file `submission.csv` with columns `id` and `essay`.

In [4]:
# For added variability, use the row index as a seed
def generate_seeded_essay(row):
    return generate_advanced_essay(row['topic'], seed=row.name)

test_df['essay'] = test_df.apply(generate_seeded_essay, axis=1)

# Save the final submission file
test_df[['id', 'essay']].to_csv("submission.csv", index=False)

print("Advanced submission file created: submission.csv")

Advanced submission file created: submission.csv
